Інсталюємо датасет кішкі проти собачевських

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"boomcrouser","key":"9806d26c109d290b0332410399df4c9d"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d salader/dogs-vs-cats
!unzip dogs-vs-cats.zip

Імпортуємо залежності

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
  
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import image_dataset_from_directory

Створюємо функцію переводу тензора до нового виду

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

Вказуємо шлях до зображень на яких буде відбуватись тренування та тестування

In [ ]:
base_dir = '/content/dogs_vs_cats/train'
test_dir = '/content/dogs_vs_cats/test'

Генеруємо датасет з папки із зображеннями та використовуємо функцію "process" до кожного елементу датасету

In [ ]:
train_datagen = image_dataset_from_directory(base_dir, image_size=(150,150), seed = 1, batch_size= 128, label_mode='binary', color_mode='grayscale')
train_datagen = train_datagen.map(process)

test_datagen = image_dataset_from_directory(test_dir, image_size=(150,150), seed = 1, batch_size= 128, label_mode='binary', color_mode='grayscale')
test_datagen = test_datagen.map(process)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


Створюємо модель із декількох шарів

In [ ]:
model = tf.keras.models.Sequential([
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(512, activation='LeakyReLU'),
    layers.Dense(64, activation='LeakyReLU'),
    layers.Dense(1, activation='sigmoid')
])

Ініціалізуємо змінну із функцією оптимізації та компілюємо модель

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

Тренеруємо модель

In [ ]:
history = model.fit(train_datagen, epochs=5, use_multiprocessing = True, shuffle = True, validation_data=test_datagen)

Epoch 1/5
157/157 [==============================] - 171s 1s/step - loss: 1.0500 - accuracy: 0.5354 - val_loss: 0.6729 - val_accuracy: 0.5696
Epoch 2/5
157/157 [==============================] - 132s 836ms/step - loss: 0.6887 - accuracy: 0.5623 - val_loss: 0.6639 - val_accuracy: 0.5984
Epoch 3/5
157/157 [==============================] - 134s 847ms/step - loss: 0.6798 - accuracy: 0.5728 - val_loss: 0.6717 - val_accuracy: 0.5786
Epoch 4/5
157/157 [==============================] - 135s 855ms/step - loss: 0.6646 - accuracy: 0.5982 - val_loss: 0.6707 - val_accuracy: 0.5862
Epoch 5/5
157/157 [==============================] - 150s 950ms/step - loss: 0.6588 - accuracy: 0.6061 - val_loss: 0.6637 - val_accuracy: 0.5968
